In [ ]:
import os 
import sys
import pickle 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time 

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

sys.path.append("../src")
from load import *
from util import * 
from pegasos import *

%matplotlib inline

In [ ]:
plt.rcParams['font.size'] = 14
plt.rcParams['axes.grid'] = True
plt.rcParams['figure.figsize'] = (10,10)

# Problem 2

The kernel matrix is given by $K = XX^T \in \mathbf{R}^{n\times n}$. Explicitly in terms of training vectors it's given by 

$$ K = 
\begin{bmatrix}
x_1^Tx_1 & x_1^Tx_2 & \cdots & x_1^Tx_n\\
x_2^Tx_1 & x_2^Tx_2 & \cdots & x_2^Tx_n\\
\vdots & \vdots & \ddots & \vdots\\
x_n^Tx_1 & x_n^Tx_2 & \cdots & x_n^Tx_n^T
\end{bmatrix}
$$

The squared Euclidean distance between two vectors is given by $d(x,y) = ||x_i-x_j||^2 = ||x_i||^2 + ||x_j||^2 - 2x_i^Tx_j$. It's clear the Gram matrix contains all the information needed to compute the pairwise distances between training examples.

# Problem 3

Consider the regularized least squares objective

$$J(w) = ||Xw-y||^2+ \lambda ||w||^2$$
where $\lambda > 0$

This problem can be written as an ordinary least squres problem of the form $\min ||Aw - b||^2$ where 

$$ 
A = \begin{bmatrix}
X\\
\sqrt{\lambda} I
\end{bmatrix},
b=
\begin{bmatrix}
y \\
0
\end{bmatrix}
$$

This is easily solved as 
$$
\begin{align*}
w^\star &= (A^TA)^{-1}A^Tb\\
&= \left(\begin{bmatrix}
X\\
\sqrt{\lambda} I
\end{bmatrix}^T
\begin{bmatrix}
X\\
\sqrt{\lambda} I
\end{bmatrix}\right)^{-1}
\begin{bmatrix}
X\\
\sqrt{\lambda} I
\end{bmatrix}^T
\begin{bmatrix}
y \\
0
\end{bmatrix}\\
&=(X^TX + \lambda I)^{-1}X^Ty
\end{align*}
$$

The normal equations for the regularized least squares problem is

$$
\begin{align*}
(X^TX + \lambda I)w^\star &= X^Ty\\
w^\star &= \frac{1}{\lambda} (X^Ty - X^TXw^\star)\\
w^\star &= \frac{1}{\lambda} X^T(y - Xw^\star)\\
         &= X^T\alpha\qquad                //\ \alpha := \frac{1}{\lambda} (y - Xw)
\end{align*}
$$

It is evident that the optimal weight vector $w^\star$ is in the span of the data since by the above it is given as a linear combination of the training vectors

$$w^\star = \sum_{i=1}^n \alpha_i x_i$$

The value of the weight vector is 
$$
\begin{align*}
\alpha &= 1/\lambda(y - Xw^\star)\\
\alpha &= 1/\lambda(y - XX^T\alpha)\\
\lambda \alpha &= y - XX^T\alpha\\
(XX^T + \lambda I)\alpha &= y\\
\alpha &= (XX^T + \lambda I)^{-1}y
\end{align*}
$$

The prediction of kernelized ridge regression on the training data is given by

$$
\begin{align*}
\hat{y} &= Xw^\star\\
&= X(X^T\alpha)\\
&= XX^T(XX^T + \lambda I)^{-1}y\\
&= K(K + \lambda I)^{-1}y
\end{align*}
$$

For the prediction of a new training example we have 
$$
\begin{align*}
f(x) &= x^Tw^\star\\
&= \sum_{i=1}^n \alpha_i x^Tx_i\\
&= \sum_{i=1}^n \alpha_i k(x, x_i)\\
&= \alpha^T k_x\\
\end{align*}
$$

# Problem 4

The loss function the SVM is given by 
$$J(w) = \frac{\lambda}{2} ||w||^2 + \frac{1}{n}\sum_{i=1}^n \ell_i(w)$$
Define $J_i(w) = \frac{\lambda}{2} ||w||^2 + \ell_i(w)$

Defining $g_i(w) \in \partial J_i(w)$ and $v_i(w) \in \partial \ell_i(w)$ by taking a subgradient of $J_i(w)$ we have $\lambda w + v_i(w) \in \partial J_i(w)$

The expected value of a subgradient of $J_i(w)$ is

\begin{align*}
    \mathbf{E}[g_i(w)] &= \sum_{i=1}^N p(i)g_i(w)\\
    &= \frac{1}{N}\sum_{i=1}^N g_i(w)\\
    &\in \partial J(w)
\end{align*}

Initializing the Pegasos algorithm with $w^{(1)} = 0$ in the $t$th step we have $w^{(t + 1)} = w^{(t)} - \eta^{(t)}g_i(w^{(t)})$. We also denote $v^{(t)} = v_i(w^{(t)})$ and want to prove the update rule $w^{(t+1)} = - \frac{1}{\lambda t} \sum_{\tau=1}^t v^{(\tau)}$

Computing $w^{(2)}$ we have 
$$
\begin{align*}
    w^{(2)} &= w^{(1)} - \eta^{(t)} g_i(w^{(1)})\\
    &= -\frac{1}{\lambda t} v_i(w^{(1)})\\
    &= -\frac{1}{\lambda t} v^{(1)}\\
\end{align*}
$$
 
This shows that the rule holds for $t=2$. Now suppose it holds for all $t$ up to $t+1$ then 

$$
\begin{align*}
    w^{(t+1)} &= w^{(t)} - \eta^{(t)} g_i(w^{(t)})\\
            &= w^{(t)} - \frac{1}{\lambda t} g_i(w^{(t)})\\
            &= w^{(t)} - \frac{1}{\lambda t} (\lambda w^{(t)} + v_i(w^{(t)}))\\
            &= \left(\frac{t-1}{t}\right) w^{(t)} - \frac{1}{\lambda t}v_i(w^{(t)})\\
            &= \left(\frac{t-1}{t}\right) \left(- \frac{1}{\lambda (t-1)} \sum_{\tau=1}^{t-1} v^{(\tau)}\right) - \frac{1}{\lambda t} v_i(w^{(t)})\qquad //\ \text{induction}\\
            &= -\frac{1}{\lambda t} \left(\sum_{\tau=1}^{t-1} v^{(\tau)}\right) - \frac{1}{\lambda t} v^{(t)}\\
            &=-\frac{1}{\lambda t} \sum_{\tau=1}^{t} v^{(\tau)}
\end{align*}
$$
which concludes the proof.

Defining $\theta^{(t+1)} := \sum_{\tau=1}^{t-1} v^{(t)}$ then we can update $w^{(t+1)} := -\frac{1}{\lambda t} \theta^{(t+1)}$. The Pegasos implementation would require that we only update $\theta$ which requires $\mathbf{nnz}(x_j)$ operations. To get $w$ we simply multiply by $-\frac{1}{\lambda t}$ before returning

# Appendix A

If $\Sigma \succeq 0$ with eigenvector $v$ with eigenvalue $\sigma$ then 
$$
\begin{align*}
v^T\Sigma v &\ge 0 \qquad //\text{definition of PSD} \\
v^T (\sigma v) &\ge 0 \\
\sigma ||v||^2 &\ge 0\\
\sigma &\ge 0
\end{align*}
$$

This shows that any eigenvector of a PSD matrix will be nonnegative

If $\Sigma \succeq 0 \implies \Sigma = BB^T$. This follows from the fact that a symmetric matrix has eigenvalue decomposition $\Sigma = Q\mathbf{diag}(\sigma_1, \sigma_2, \ldots, \sigma_n)Q^T$. Since all the eigenvalues are nonnegative we have $\Sigma = QD^{1/2}D^{T/2}Q^T$ where $D^{1/2} = \mathbf{diag}\left(\sqrt{\sigma_1}, \sqrt{\sigma_2}, \ldots, \sqrt{\sigma_n}\right)$. We can identify $B= QD^{1/2}$

If $\Sigma = BB^T \implies \Sigma\succeq 0$. This follows immediately from the definition of PSD

$$
\begin{align*}
x^T\Sigma x &= x^TBB^Tx\\
  &= (Bx)^TB^Tx\\
  &= ||Bx||^2 \qquad //\ x^Tx = ||x||^2\\
  &\ge 0
\end{align*}
$$

Therefore $\Sigma \succeq 0$ if and only if it has a square root.

# Appendix B

If $\Sigma \succ 0$ with eigenvector $v$ and eigenvalue $\sigma$ the 
$$
\begin{align*}
    v^T\Sigma v > 0  \qquad //\ \text{def of positive definite}\\
    v^T(\sigma v) > 0\\
    \sigma||v||^2 > 0\\
    \sigma > 0
\end{align*}
$$

By the spectral theorem we can write $\Sigma = QDQ^T$ where $Q^TQ = I$ and $D = \mathbf{diag}(\sigma_1,\sigma_2,\ldots,\sigma_n)$. We can trivially verify that $QD^{-1}Q^T$ is the inverse of $\Sigma$ since 

$$
\begin{align*}
    (QD^{-1}Q^T)\Sigma &= (QD^{-1}Q^T)QDQ^T \\
    &= QD^{-1}DQ^T\\
    &=QQ^T\\
    &= I \qquad \blacksquare
\end{align*}
$$

If $M \succeq 0$ and $\lambda > 0$ then $M+\lambda I \succ 0$.

$$
\begin{align*}
  x^T(M+\lambda I)x &= x^TMx +\lambda ||x||^2\\
  &\ge \lambda ||x||^2 \\
  & > 0      \qquad \forall x \ne 0
\end{align*}
$$

If $M \succeq 0$ and $N\succ 0$ then $M+N \succ 0$ which follows immediately form the definition

$$
\begin{align*}
    x^T(M+N)x &= x^TMx + x^TNx\\
    &\ge x^TNx\\
    &> 0 \qquad \forall x \ne 0
\end{align*}
$$